# Criar VENV

https://oandersonbm.medium.com/usando-python-no-vs-code-com-ambiente-virtual-venv-ecef7959b652

In [48]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import locale

# Definindo a configuração local para o Brasil (pt_BR)
locale.setlocale(locale.LC_ALL, 'pt_BR')

# Personalizando para ver todas as colunas do DataFrame
pd.set_option('display.max_columns', None)

# Base de dados

In [49]:
# Realizando a junção dos 3 arquivos .CSV num único DataFrame

# Caminhos para os arquivos .CSV
caminho_1 = r"C:\Users\User\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini.csv"

caminho_2 = r"C:\Users\User\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini(1).csv"

caminho_3 = r"C:\Users\User\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini(2).csv"

caminho_4 = r"C:\Users\User\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini(3).csv"


# Leitura dos arquivos .CSV em DataFrames individuais
df1 = pd.read_csv(caminho_1)
df2 = pd.read_csv(caminho_2)
df3 = pd.read_csv(caminho_3)
df4 = pd.read_csv(caminho_4)

# Concatenação dos DataFrames em um único DataFrame
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Excluindo linhas que contenham a string "Tela" em qualquer coluna
df = df[~df.apply(lambda row: row.astype(str).str.contains('Tela')).any(axis=1)]

# Transformando o tipo de dado das colunas de valores monetários
df['A_LIQUIDAR'] = df['A_LIQUIDAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['LIQUIDADO_A_PAGAR'] = df['LIQUIDADO_A_PAGAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['TOTAL_A_PAGAR'] = df['TOTAL_A_PAGAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['PAGO'] = df['PAGO'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))

# Convertendo a coluna DATA para datetime
df['DATA'] = pd.to_datetime(df['DATA'], format='%d/%m/%y')

# Convertendo a coluna DIAS em número inteiro
df['DIAS'] = df['DIAS'].astype(int)



# Trabalhando com um DATAFRAME voltado para controle dos SALDOS

In [50]:
df_saldos = df[['SIGLA UG', 'ANO', 'NOME_CREDOR', 'DATA', 'DIAS', 'ND','NDSI', 'NOME_NDSI', 'NE', 'PI', 'NOME_PI','TIPO', 'NOME_UGR', 'A_LIQUIDAR', 'LIQUIDADO_A_PAGAR', 'TOTAL_A_PAGAR', 'PAGO']]

In [51]:
# Saldos Totais
total_pago = df['PAGO'].sum().round(2)
total_a_liquidar = df['A_LIQUIDAR'].sum().round(2)
total_liquidado_a_pagar = df['LIQUIDADO_A_PAGAR'].sum().round(2)
total_a_pagar = df['TOTAL_A_PAGAR'].sum().round(2)


print(f'O total pago pela ESA até o momento foi: {locale.currency(total_pago, grouping=True)}')
print(f'O total de saldo a liquidar pela ESA até o momento foi: {locale.currency(total_a_liquidar, grouping=True)}')
print(f'O total de saldo liquidado a pagar pela ESA até o momento foi: {locale.currency(total_liquidado_a_pagar, grouping=True)}')
print(f'O total de saldo a pagar pela ESA até o momento foi: {locale.currency(total_a_pagar, grouping=True)}')

O total pago pela ESA até o momento foi: R$ 25.908.873,47
O total de saldo a liquidar pela ESA até o momento foi: R$ 6.317.926,85
O total de saldo liquidado a pagar pela ESA até o momento foi: R$ 108.647,95
O total de saldo a pagar pela ESA até o momento foi: R$ 6.426.574,80


In [52]:
# Saldo a liquidar por ND
cols = ['ND', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('ND').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='ND', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ND',
             labels={'ND':'Natureza da Despesa', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=500,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [53]:
# Saldo a liquidar por Subitem
cols = ['NDSI', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NDSI').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NDSI', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por NDSI',
             labels={'NDSI':'Subitem da despesa', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=600,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [54]:
# Saldo a liquidar por tipo de PI
cols = ['NOME_PI', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NOME_PI').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NOME_PI', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por PI',
             labels={'NOME_PI':'Tipo de PI', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=800,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [55]:
# Saldo a liquidar por tipo de Empenho
cols = ['TIPO', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('TIPO').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='TIPO', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por tipo de empenho',
             labels={'TIPO':'Tipo de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [56]:
# Saldo a liquidar por ano
cols = ['ANO', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('ANO').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='ANO', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'ANO':'Ano do Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [57]:
# Saldo a liquidar por tipo de NE
cols = ['TIPO_NE', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('TIPO_NE').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='TIPO_NE', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'TIPO_NE':'Tipo de Nota de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [58]:
# Saldo a liquidar por tipo de NE
cols = ['NOME_UGR', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NOME_UGR').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NOME_UGR', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'NOME_UGR':'Nome das UGR', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [59]:
# Saldo a liquidar por ND
cols = ['ND', 'A_LIQUIDAR', 'LIQUIDADO_A_PAGAR', 'PAGO']

df_aux = df.loc[:, cols].groupby('ND').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
# Criando o gráfico
fig = go.Figure()

# Adicionando as colunas y1, y2 e y3 no eixo y
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['A_LIQUIDAR'], name='A Liquidar'))
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['LIQUIDADO_A_PAGAR'], name='Liquidado a pagar'))
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['PAGO'], name='Pago'))

fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")
fig.update_yaxes(type="log")

fig.show()

In [60]:
df[(df['ACAO'] == '212O') & df['A_LIQUIDAR'] != 0]

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
468,160129,ESA,212O,MOVIMENTACAO DE MILITARES,2023,03849500000190,A. CENTRAL TRANSPORTES LTDA,2023-07-10,106,160533,4º CGCFEX,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2023NE000863,D6DAATSLOCO,ATS - ATIVIDADES SETORIAIS,6012,171389,ANO,ESTIMATIVO,ESCOLA DE SARGENTOS DAS ARMAS,PR,MG,160505,DGP,D6DA,ATS,LOCO,15000.00,0.0,15000.00,0.0
469,160129,ESA,212O,MOVIMENTACAO DE MILITARES,2023,03849500000190,A. CENTRAL TRANSPORTES LTDA,2023-07-24,92,160533,4º CGCFEX,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2023NE000888,D6DAATSLOCO,ATS - ATIVIDADES SETORIAIS,6012,171389,ANO,ESTIMATIVO,ESCOLA DE SARGENTOS DAS ARMAS,PR,MG,160505,DGP,D6DA,ATS,LOCO,22455.00,0.0,22455.00,0.0
470,160129,ESA,212O,MOVIMENTACAO DE MILITARES,2023,03849500000190,A. CENTRAL TRANSPORTES LTDA,2023-07-25,91,160533,4º CGCFEX,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2023NE000897,D6DAATSLOCO,ATS - ATIVIDADES SETORIAIS,6012,171389,ANO,ESTIMATIVO,ESCOLA DE SARGENTOS DAS ARMAS,PR,MG,160505,DGP,D6DA,ATS,LOCO,18600.00,0.0,18600.00,0.0
471,160129,ESA,212O,MOVIMENTACAO DE MILITARES,2023,03849500000190,A. CENTRAL TRANSPORTES LTDA,2023-07-25,91,160533,4º CGCFEX,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2023NE000898,D6DAATSLOCO,ATS - ATIVIDADES SETORIAIS,6012,171389,ANO,ESTIMATIVO,ESCOLA DE SARGENTOS DAS ARMAS,PR,MG,160505,DGP,D6DA,ATS,LOCO,9000.00,0.0,9000.00,0.0
472,160129,ESA,212O,MOVIMENTACAO DE MILITARES,2023,03849500000190,A. CENTRAL TRANSPORTES LTDA,2023-08-08,77,160533,4º CGCFEX,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2023NE000960,D6DAATSLOCO,ATS - ATIVIDADES SETORIAIS,6012,171389,ANO,ESTIMATIVO,ESCOLA DE SARGENTOS DAS ARMAS,PR,MG,160505,DGP,D6DA,ATS,LOCO,24950.00,0.0,24950.00,0.0
473,160129,ESA,212O,MOVIMENTACAO DE MILITARES,2023,03849500000190,A. CENTRAL TRANSPORTES LTDA,2023-08-11,74,160533,4º CGCFEX,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2023NE000989,D6DAATSLOCO,ATS - ATIVIDADES SETORIAIS,6012,171389,ANO,ESTIMATIVO,ESCOLA DE SARGENTOS DAS ARMAS,PR,MG,160505,DGP,D6DA,ATS,LOCO,10800.00,0.0,10800.00,0.0
474,160129,ESA,212O,MOVIMENTACAO DE MILITARES,2023,03849500000190,A. CENTRAL TRANSPORTES LTDA,2023-08-11,74,160533,4º CGCFEX,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2023NE000990,D6DAATSLOCO,ATS - ATIVIDADES SETORIAIS,6012,171389,ANO,ESTIMATIVO,ESCOLA DE SARGENTOS DAS ARMAS,PR,MG,160505,DGP,D6DA,ATS,LOCO,28443.00,0.0,28443.00,0.0
475,160129,ESA,212O,MOVIMENTACAO DE MILITARES,2023,03849500000190,A. CENTRAL TRANSPORTES LTDA,2023-08-16,69,160533,4º CGCFEX,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2023NE001018,D6DAATSLOCO,ATS - ATIVIDADES SETORIAIS,6012,171389,ANO,ESTIMATIVO,ESCOLA DE SARGENTOS DAS ARMAS,PR,MG,160505,DGP,D6DA,ATS,LOCO,9900.00,0.0,9900.00,0.0
476,160129,ESA,212O,MOVIMENTACAO DE MILITARES,2023,03849500000190,A. CENTRAL TRANSPORTES LTDA,2023-08-23,62,160533,4º CGCFEX,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2023NE001059,D6DAATSLOCO,ATS - ATIVIDADES SETORIAIS,6012,171389,ANO,ESTIMATIVO,ESCOLA DE SARGENTOS DAS ARMAS,PR,MG,160505,DGP,D6DA,ATS,LOCO,14400.00,0.0,14400.00,0.0
477,160129,ESA,212O,MOVIMENTACAO DE MILITARES,2023,03849500000190,A. CENTRAL TRANSPORTES LTDA,2023-08-29,56,160533,4º CGCFEX,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2023NE001079,D6DAATSLOCO,ATS - ATIVIDADES SETORIAIS,6012,171389,ANO,ESTIMATIVO,ESCOLA DE SARGENTOS DAS ARMAS,PR,MG,160505,DGP,D6DA,ATS,LOCO,11160.00,0.0,11160.00,0.0


# Suporte para o Dashboard

In [61]:
cols = ['NE', 'A_LIQUIDAR', 'DIAS', 'NOME_NDSI', 'NDSI']

df_aux = df.loc[:, cols].sort_values(by='DIAS', ascending=False)
df_aux = df_aux[df_aux['A_LIQUIDAR'] != 0].head(10)
df_aux

,NE,A_LIQUIDAR,DIAS,NOME_NDSI,NDSI
1118,2022NE000742,180.00,476,LOCACAO BENS MOV. OUT.NATUREZAS E INTANGIVEIS,33903914
1109,2022NE000788,6334.69,467,OUTSOURCING DE IMPRESSAO,33904016
1137,2022NE001086,3466.70,383,MANUTENCAO E CONSERVACAO DE EQUIPAMENTOS DE TIC,33904012
1136,2022NE001085,5543.35,383,MANUT. E CONSERV. DE MAQUINAS E EQUIPAMENTOS,33903917
1103,2022NE001087,2397.00,383,MANUTENCAO E CONSERVACAO DE EQUIPAMENTOS DE TIC,33904012
1385,2022NE001843,1016.25,328,SERVICOS GRAFICOS E EDITORIAIS,33903963
1126,2022NE002421,5758.75,328,SERVICOS GRAFICOS E EDITORIAIS,33903963
931,2022NE002482,155796.97,314,MANUTENCAO E CONSERV. DE BENS IMOVEIS,33903916
932,2022NE002483,263051.08,314,MANUTENCAO E CONSERV. DE BENS IMOVEIS,33903916
1123,2022NE002703,146.93,300,PASSAGENS PARA O PAIS,33903301


In [62]:
# Saldo a liquidar por tipo de NE
cols = ['NE', 'A_LIQUIDAR', 'DIAS', 'NOME_NDSI', 'NDSI']
df_aux = df.loc[:, cols].sort_values(by='DIAS', ascending=False)
df_aux = df_aux[df_aux['A_LIQUIDAR'] != 0].head(10)
fig = px.bar(df_aux, 
             x='NE', 
             y='A_LIQUIDAR',
             title='10 empenhos com mais tempo na tela',
             labels={'NE':'Nota de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             color='DIAS',
             color_continuous_scale = 'reds',
             log_y=True,
             template='plotly_dark',
             hover_name="NE", hover_data={'DIAS': True, 'A_LIQUIDAR': True, 'NOME_NDSI': True, 'NDSI': True})
fig.update_traces(textposition = 'outside')


fig.show()

## Informações para SideBar

In [63]:
# Qtde de empenhos realizados até o momento
df.shape[0]

3165

# Requisição em API para consulta a CNPJ

In [64]:
import requests
from pprint import pprint

url = "https://receitaws.com.br/v1/cnpj/21331404000138"

headers = {"Accept": "application/json"}

response = requests.get(url, headers=headers)

pprint(response.json())

{'abertura': '03/11/2014',
 'atividade_principal': [{'code': '79.11-2-00', 'text': 'Agências de viagens'}],
 'atividades_secundarias': [{'code': '59.11-1-99',
                             'text': 'Atividades de produção cinematográfica, '
                                     'de vídeos e de programas de televisão '
                                     'não especificadas anteriormente'},
                            {'code': '63.99-2-00',
                             'text': 'Outras atividades de prestação de '
                                     'serviços de informação não especificadas '
                                     'anteriormente'},
                            {'code': '79.12-1-00',
                             'text': 'Operadores turísticos'},
                            {'code': '79.90-2-00',
                             'text': 'Serviços de reservas e outros serviços '
                                     'de turismo não especificados '
                                    

In [65]:
def consulta_cnpj(numero_cnpj):
    url = f"https://receitaws.com.br/v1/cnpj/{numero_cnpj}"

    headers = {"Accept": "application/json"}

    response = requests.get(url, headers=headers)
    
    dic_fornecedor = response.json()
    
    return dic_fornecedor

# Criar um dicionário vazio fora da função
dicionario = {}

# Chamar a função e atribuir o resultado ao dicionário
dicionario = consulta_cnpj('81243735001977')

# Imprimir o dicionário
pprint(dicionario)

{'abertura': '28/08/2015',
 'atividade_principal': [{'code': '26.21-3-00',
                          'text': 'Fabricação de equipamentos de informática'}],
 'atividades_secundarias': [{'code': '26.22-1-00',
                             'text': 'Fabricação de periféricos para '
                                     'equipamentos de informática'},
                            {'code': '27.59-7-99',
                             'text': 'Fabricação de outros aparelhos '
                                     'eletrodomésticos não especificados '
                                     'anteriormente, peças e acessórios'},
                            {'code': '46.19-2-00',
                             'text': 'Representantes comerciais e agentes do '
                                     'comércio de mercadorias em geral não '
                                     'especializado'},
                            {'code': '46.51-6-01',
                             'text': 'Comércio atacadista de equipam

In [66]:
pprint(dicionario)

{'abertura': '28/08/2015',
 'atividade_principal': [{'code': '26.21-3-00',
                          'text': 'Fabricação de equipamentos de informática'}],
 'atividades_secundarias': [{'code': '26.22-1-00',
                             'text': 'Fabricação de periféricos para '
                                     'equipamentos de informática'},
                            {'code': '27.59-7-99',
                             'text': 'Fabricação de outros aparelhos '
                                     'eletrodomésticos não especificados '
                                     'anteriormente, peças e acessórios'},
                            {'code': '46.19-2-00',
                             'text': 'Representantes comerciais e agentes do '
                                     'comércio de mercadorias em geral não '
                                     'especializado'},
                            {'code': '46.51-6-01',
                             'text': 'Comércio atacadista de equipam

In [67]:
dicionario['nome']

'POSITIVO TECNOLOGIA S.A.'

In [68]:
dicionario['telefone']

'(41) 3316-7700'

In [69]:
dicionario['email']

'fiscal@positivo.com.br'

In [70]:
dicionario['logradouro'] + ', ' + dicionario['numero'] + ' - ' + dicionario['bairro'] + ' - ' + dicionario['municipio'] + ' / ' + dicionario['uf']

'R JAVARI, 1255 - DISTRITO INDUSTRIAL I - MANAUS / AM'

In [71]:
# filtrando dos dados da coluna FAV
df_cnpj = df[df['NOME_PI'] != 'PROFISSIONAL DE SAUDE AUTONOMO']
df_cnpj = df[df['NOME_PI'] != 'ATENDIMENTO MEDICO-HOPITALR/FC']
df_cnpj = df[df['NOME_PI'] != 'IND-INDENIZACOES']
df_cnpj = df[df['NOME_PI'] != 'IND - INDENIZACOES']
df_cnpj = df[df['NOME_PI'] != 'INDENIZACAO DESPESAS.']
df_cnpj = df[df['NOME_PI'] != 'AUXILIO FINANCEIRO NÃO INDENIZÁVEL']
df_cnpj = df[df['NOME_PI'] != 'INRE - RESTITUICAO E RESSARCIMENTO']
df_cnpj = df[df['NOME_NDSI'] != 'OUTROS SERV.DE TERCEIROS PJ- PAGTO ANTECIPADO']
df_cnpj = df[df['CREDOR'] != '160129']

df_cnpj['CREDOR'].unique()

array(['33658130000175', '01590728000850', '05872523000141',
       '06981180000116', '09583781000169', '12742245000173',
       '17281106000103', '20584944000160', '27325275000151',
       '27995686000154', '34028316001509', '39319890000134',
       '41505690000118', '41776201000162', '43003732000148',
       '43433811000199', '44812602000119', '45424853000199',
       '76535764000143', '00139736000145', '01255291000121',
       '01804159000121', '02338743000100', '02347121000210',
       '03157268000120', '03539933000140', '04785664000164',
       '05088041000103', '06022475000165', '06894510000136',
       '07027329743', '07457410000296', '07912053000128',
       '08605233000120', '08703140000138', '09035549000196',
       '09249668000141', '09505477000101', '10463704000154',
       '10526703000101', '10842829000195', '10902067000175',
       '11094173000132', '11532912000120', '12669334000131',
       '13857945000176', '13986656000177', '14459158000139',
       '14807420000199', '1

In [72]:
df_cnpj[df_cnpj['CREDOR'] == '160129']

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO


In [73]:
ano_atual = str(datetime.now().year)


df_corrente = df[df['ANO'] == ano_atual]
df_corrente

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,160129,ESA,156M,MODERNIZACAO E TRANSFORMACAO ESTRATEG,2023,33658130000175,FIEL COMERCIAL E SERVICOS LTDA,2023-10-10,14,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903916,MANUTENCAO E CONSERV. DE BENS IMOVEIS,2023NE001252,FIL311XARTC,CONSTRUIR E ADEQUAR INST SIST EDU E CULT EB,6012,215822,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,SP,MG,160502,DEC-GESTOR,FIL3,11X,ARTC,300000.0,0.0,300000.0,0.0
32,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,00139736000145,TECELAGEM SAO DOMINGOS LTDA,2023-09-12,42,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903022,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,2023NE001136,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160073,DGO,I3DA,FUN,ADOM,0.0,0.0,0.0,450.0
33,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,01255291000121,H.A DISTRIBUIDORA LTDA,2023-04-11,196,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903022,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,2023NE000361,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,"ESCOLA DE SARGENTOS DAS ARMAS, AVENIDA SETE DE...",MG,MG,160073,DGO,I3DA,FUN,ADOM,0.0,0.0,0.0,2646.0
34,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,01255291000121,H.A DISTRIBUIDORA LTDA,2023-06-22,124,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903022,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,2023NE000737,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160073,DGO,I3DA,FUN,ADOM,0.0,0.0,0.0,4647.0
35,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,01255291000121,H.A DISTRIBUIDORA LTDA,2023-07-31,85,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903022,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,2023NE000930,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160073,DGO,I3DA,FUN,ADOM,0.0,0.0,0.0,2211.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,65095531000150,LOGOS PAPELARIA LTDA,2023-05-25,152,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905245,EQUIPAMENTOS DE TIC - IMPRESSORAS,2023NE001044,E3PCFSCINFO,INFORMATICA,6012,215839,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,167504,COEX - GESTOR,E3PC,FSC,INFO,0.0,0.0,0.0,1800.0
3164,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,81243735000903,POSITIVO TECNOLOGIA S.A.,2023-02-15,251,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2023NE000261,E3PCFSCINFO,INFORMATICA,6012,215839,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,BA,MG,167504,COEX - GESTOR,E3PC,FSC,INFO,0.0,0.0,0.0,7114.0
3165,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,98488040644,MARIO HENRIQUE CUBBI,2023-05-22,155,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903001,COMBUSTIVEIS E LUBRIFICANTES AUTOMOTIVOS,2023NE001035,E3PCFSCCOLU,COMBUSTIVEIS E LUBRIFICANTES,6012,215839,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,167504,COEX - GESTOR,E3PC,FSC,COLU,0.0,0.0,0.0,2236.2
3166,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,98488040644,MARIO HENRIQUE CUBBI,2023-08-31,54,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903096,MATERIAL DE CONSUMO - PAGTO ANTECIPADO,2023NE001506,E3PCOPFCOLU,COMBUSTIVEL E LUBRIFICANTES,6012,215839,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,167504,COEX - GESTOR,E3PC,OPF,COLU,0.0,0.0,0.0,675.0


In [74]:
df['ANO'].unique()

array(['2023', '2022', '2021'], dtype=object)

# Pesquisas no DataFrame

In [85]:
# Criando um DataFrame para testes 
df_teste = df.copy()

## Empenhos com mais de 100 dias

### Empenhos da ND 30

In [76]:
# Empenhos da ND 30 com prazo de entrega superior a 100 dias

df_nd30_100dias = df_teste[(df_teste['A_LIQUIDAR'] != 0) & (df_teste['ND'] == '339030') & (df_teste['DIAS'] > 100) & (df_teste['TIPO_NE'] == 'ORDINARIO') ].sort_values(by='DIAS', ascending=False).reset_index(drop=True)

df_nd30_100dias.sort_values(by='A_LIQUIDAR', ascending=False).reset_index(drop=True)

#df_nd30_100dias.to_excel('nd30_PA.xlsx')

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,160129,ESA,2865,SUPRIMENTO DE FARDAMENTO,2023,02246382000163,PALMILHADO INDUSTRIA E COMERCIO LTDA,2023-05-18,159,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903023,"UNIFORMES, TECIDOS E AVIAMENTOS",2023NE000525,E6MIPLJUESC,MATERIAL DE INTENDÊNCIA,0032,171388,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160504,COEX/GESTOR,E6MI,PLJ,UESC,17975.00,0.0,17975.00,0.0
1,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,20402517000114,DHZ COMERCIO DE SUPRIMENTOS LTDA,2023-07-11,105,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903017,MATERIAL DE TIC - MATERIAL DE CONSUMO,2023NE000866,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,15525.90,0.0,15525.90,0.0
2,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,13754000000129,"G4 SERVICO E COMERCIO DE PRODUTOS INDUSTRIAIS,...",2023-06-02,144,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903039,MATERIAL P/ MANUTENCAO DE VEICULOS,2023NE001125,E3PCFSCMABM,MANUTENCAO DE BENS MOVEIS,6012,215839,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,RJ,MG,167504,COEX - GESTOR,E3PC,FSC,MABM,9999.67,0.0,9999.67,0.0
3,160129,ESA,212B,BENEFICIOS OBRIGATORIOS AOS SERVIDORE,2023,28259514000185,CLEVER FERREIRA COSTA 01175363693,2023-05-03,174,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903021,MATERIAL DE COPA E COZINHA,2023NE000465,E6SUPLJA5PA,A5PA - PASA,0032,171397,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,MG,MG,160504,COEX/GESTOR,E6SU,PLJ,A5PA,3044.75,0.0,3044.75,0.0
4,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,27325275000151,R F DEMORE SERVICOS,2023-06-23,123,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903022,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,2023NE000755,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SP,MG,160073,DGO,I3DA,FUN,ADOM,2309.00,0.0,2309.00,0.0
5,160129,ESA,212B,BENEFICIOS OBRIGATORIOS AOS SERVIDORE,2023,05291541000130,TY BORTHOLIN COMERCIAL LTDA,2023-05-03,174,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903019,MATERIAL DE ACONDICIONAMENTO E EMBALAGEM,2023NE000453,E6SUPLJA5PA,A5PA - PASA,0032,171397,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,SP,MG,160504,COEX/GESTOR,E6SU,PLJ,A5PA,1956.20,0.0,1956.20,0.0
6,160129,ESA,21A0,APRESTAMENTO DAS FORCAS,2023,11718643000191,L M C PAES,2023-05-10,167,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903039,MATERIAL P/ MANUTENCAO DE VEICULOS,2023NE000497,E5MBSUNPREV,SUPRIMENTO E MANUTENCAO MOTOMECANIZACAO_BLIND,6012,171470,ANO,ORDINARIO,"ESCOLA DE SARGENTOS DAS ARMAS, AVENIDA SETE DE...",RJ,MG,160504,COEX/GESTOR,E5MB,SUN,PREV,1713.30,0.0,1713.30,0.0
7,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,09583781000169,2 CLICK SOLUCOES LTDA,2023-06-06,140,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903022,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,2023NE000635,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,GO,MG,160073,DGO,I3DA,FUN,ADOM,840.00,0.0,840.00,0.0
8,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,02852112698,LUCIANO ALVES DE PAULA,2023-06-13,133,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903096,MATERIAL DE CONSUMO - PAGTO ANTECIPADO,2023NE001191,E3PCOPFCOLU,COMBUSTIVEL E LUBRIFICANTES,6012,215839,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MS,MG,167504,COEX - GESTOR,E3PC,OPF,COLU,5.22,0.0,5.22,0.0


### Empenhos ND 52

In [77]:
df_nd52_100dias = df_teste[(df_teste['A_LIQUIDAR'] != 0) & (df_teste['ND'] == '449052') & (df_teste['DIAS'] > 50) ].sort_values(by='DIAS', ascending=False).reset_index(drop=True)

df_nd52_100dias.reset_index(drop=True)

#df_nd52_100dias.to_excel('nd52_PA.xlsx')

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,160129,ESA,212B,BENEFICIOS OBRIGATORIOS AOS SERVIDORE,2023,31669124000198,LS REFRIGERACAO LTDA,2023-04-18,189,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905228,MAQUINAS E EQUIPAMENTOS DE NATUREZA INDUSTRIAL,2023NE000392,E6SUPLJA5PA,A5PA - PASA,0032,171397,ANO,ORDINARIO,"ESCOLA DE SARGENTOS DAS ARMAS, AVENIDA SETE DE...",SC,MG,160504,COEX/GESTOR,E6SU,PLJ,A5PA,8837.42,0.0,8837.42,0.0
1,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,20362687000112,VITORIA DECORACOES LTDA,2023-05-29,148,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905251,PECAS NAO INCORPORAVEIS A IMOVEIS,2023NE000564,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,MG,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,24059.00,0.0,24059.00,0.0
2,160129,ESA,212B,BENEFICIOS OBRIGATORIOS AOS SERVIDORE,2023,45769285000168,REDNOV FERRAMENTAS LTDA.,2023-06-07,139,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905242,MOBILIARIO EM GERAL,2023NE000650,E6SUPLJA5PA,A5PA - PASA,0032,171397,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SC,MG,160504,COEX/GESTOR,E6SU,PLJ,A5PA,9441.53,0.0,9441.53,0.0
3,160129,ESA,21A0,APRESTAMENTO DAS FORCAS,2023,48241204000140,EQUIPAT - EQUIPAMENTOS E MATERIAIS ELETRICOS E...,2023-06-16,130,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905238,"MAQ., FERRAMENTAS E UTENSILIOS DE OFICINA",2023NE000668,E5MMPDRDEGE,DESCENTRALIZACAO RECURSOS MOTOMECANIZACAO,6012,171470,ANO,ORDINARIO,"ESCOLA DE SARGENTOS DAS ARMAS, AVENIDA SETE DE...",RS,MG,160504,COEX/GESTOR,E5MM,PDR,DEGE,995.00,0.0,995.00,0.0
4,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,28320324000126,FREDERICO DELGADO DE ALMEIDA,2023-06-19,127,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905224,"EQUIPAMENTO DE PROTECAO, SEGURANCA E SOCORRO",2023NE000672,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,1379.10,0.0,1379.10,0.0
5,160129,ESA,21A0,APRESTAMENTO DAS FORCAS,2023,48241204000140,EQUIPAT - EQUIPAMENTOS E MATERIAIS ELETRICOS E...,2023-06-19,127,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905212,APARELHOS E UTENSILIOS DOMESTICOS,2023NE000677,E5MMPDRDEGE,DESCENTRALIZACAO RECURSOS MOTOMECANIZACAO,6012,171470,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,RS,MG,160504,COEX/GESTOR,E5MM,PDR,DEGE,551.80,0.0,551.80,0.0
6,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,81243735001977,POSITIVO TECNOLOGIA S.A.,2023-06-26,120,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2023NE000762,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,ANO,ORDINARIO,"ESCOLA DE SARGENTOS DAS ARMAS, AVENIDA SETE DE...",AM,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,10671.00,0.0,10671.00,0.0
7,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,81243735001977,POSITIVO TECNOLOGIA S.A.,2023-06-28,118,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2023NE000781,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,AM,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,21342.00,0.0,21342.00,0.0
8,167129,ESA,2004,ASSISTENCIA MEDICA E ODONTOLOGICA AOS,2023,81243735000903,POSITIVO TECNOLOGIA S.A.,2023-07-12,104,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2023NE001312,D5SAFUSASOC,ASSISTENCIA SOCIAL,0032,215846,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,BA,MG,167505,DGP,D5SA,FUS,ASOC,15272.00,0.0,15272.00,0.0
9,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,81243735000903,POSITIVO 

### Consolidação ND 30 e ND 39

In [78]:
nd = ['449052', '339030' ]

df_foco = df_teste.loc[df_teste['ND'].isin(nd) & (df_teste['DIAS'] > 60) & (df_teste['A_LIQUIDAR'] != 0), :].sort_values(by='DIAS', ascending=False).reset_index(drop=True)

df_foco.reset_index(drop=True)

df_foco.to_excel('foco_almox.xlsx')

### Recursos Liquidados a pagar

In [79]:
df_liqui_a_pagar =df_teste[ df_teste['LIQUIDADO_A_PAGAR'] != 0 ]

df_liqui_a_pagar.sort_values(by='LIQUIDADO_A_PAGAR', ascending=False).reset_index(drop=True).head(2)

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,167129,ESA,2004,ASSISTENCIA MEDICA E ODONTOLOGICA AOS,2023,42855999000362,UNIMED TRES CORACOES COOPERATIVA DE TRAB MEDIC...,2023-10-03,21,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903950,"SERV.MEDICO-HOSPITAL.,ODONTOL.E LABORATORIAIS",2023NE001631,D8SAFUSOCSA,ORGANIZACAO CIVIL DE SAUDE,0032,215845,ANO,ESTIMATIVO,CLINICA/CONSULTÓRIO/HOSPITAL,MG,MG,167505,DGP,D8SA,FUS,OCSA,0.0,61011.43,61011.43,0.0
1,167129,ESA,2004,ASSISTENCIA MEDICA E ODONTOLOGICA AOS,2023,04648489000163,CENTRO DE CARDIOLOGIA DE TRES CORACOES LTDA,2023-08-29,56,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903950,"SERV.MEDICO-HOSPITAL.,ODONTOL.E LABORATORIAIS",2023NE001435,D8SAFCTOCSA,FATOR DE CUSTO - CONTRATO DE OBJETIVOS,0032,215842,ANO,ESTIMATIVO,CLINICA/CONSULTÓRIO/HOSPITAL,MG,MG,167505,DGP,D8SA,FCT,OCSA,0.0,7263.48,7263.48,0.0


In [80]:
total_liqui_a_pagar = df_liqui_a_pagar['LIQUIDADO_A_PAGAR'].sum()
locale.currency(total_liqui_a_pagar, grouping=True)

'R$ 108.647,95'

In [81]:
df_aux = df_liqui_a_pagar.groupby(by='NOME_NDSI')['LIQUIDADO_A_PAGAR'].sum().sort_values( ascending=False).reset_index()

fig = px.bar(df_aux,
             x = 'NOME_NDSI',
             y = 'LIQUIDADO_A_PAGAR',
             log_y=True,
             height=900,
             title='Saldos liquidados a pagar por NDSI',
             labels={'NOME_NDSI':'Descrição do Subitem', 'LIQUIDADO_A_PAGAR':'Liquidado a Pagar'},
             text_auto='.3s',
             color='LIQUIDADO_A_PAGAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [90]:
df_teste.loc[(df_teste['ACAO'] == '2000') & (df_teste['A_LIQUIDAR'] != 0), :]

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
7,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2022,05872523000141,AEF - MANUTENCAO EM EQUIPAMENTOS INDUSTRIAIS LTDA,2022-12-29,299,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903917,MANUT. E CONSERV. DE MAQUINAS E EQUIPAMENTOS,2022NE002723,I3DACNTMABM,MNT DE MÁQ E EQP ADMINISTRATIVOS,0032,171460,RPNP,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS,MG,MG,160073,DGO,I3DA,CNT,MABM,809.64,0.0,809.64,39023.43
36,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,01255291000121,H.A DISTRIBUIDORA LTDA,2023-09-12,42,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903022,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,2023NE001137,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160073,DGO,I3DA,FUN,ADOM,541.00,0.0,541.00,0.00
40,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,02347121000210,ASMAR & FURTADO LTDA,2023-03-28,210,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903946,SERVICOS DOMESTICOS,2023NE000291,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,GLOBAL,"ESCOLA DE SARGENTOS DAS ARMAS, AVENIDA SETE DE...",MG,MG,160073,DGO,I3DA,FUN,ADOM,1167.17,0.0,1167.17,3832.83
43,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,03539933000140,INDUSTRIA E COMERCIO DE RODOS RODOBEM LTDA,2023-09-12,42,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903022,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,2023NE001138,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,GO,MG,160073,DGO,I3DA,FUN,ADOM,598.00,0.0,598.00,0.00
46,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,05872523000141,AEF - MANUTENCAO EM EQUIPAMENTOS INDUSTRIAIS LTDA,2023-04-12,195,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903917,MANUT. E CONSERV. DE MAQUINAS E EQUIPAMENTOS,2023NE000371,I3DACNTMABM,MNT DE MÁQ E EQP ADMINISTRATIVOS,0032,171460,ANO,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160073,DGO,I3DA,CNT,MABM,33161.88,0.0,33161.88,16580.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472,167129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,58512096000157,TRATORTEC PECAS E SERVICOS PARA TRATORES LTDA,2023-09-27,27,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903919,MANUTENCAO E CONSERV. DE VEICULOS,2023NE001603,C1ENCONDETM,CONCURSOS AS ESCOLAS MILITARES,0032,171502,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,SP,MG,167503,DECEX - GESTOR,C1EN,CON,DETM,620.00,0.0,620.00,0.00
1477,167129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,65353401000170,COMERCIAL VENER LTDA,2023-10-03,21,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903022,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,2023NE001621,C1ENCONDETM,CONCURSOS AS ESCOLAS MILITARES,0032,171502,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,MG,MG,167503,DECEX - GESTOR,C1EN,CON,DETM,372.00,0.0,372.00,0.00
1479,167129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,81243735000903,POSITIVO TECNOLOGIA S.A.,2023-08-18,67,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2023NE001414,C1ENCONDETM,CONCURSOS AS ESCOLAS MILITARES,0032,171502,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,BA,MG,167503,DECEX - GESTOR,C1EN,CON,DETM,21864.00,0.0,21864.00,0.00
1480,167129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,81243735000903,POSITIVO TECNOLOGIA S.A.,2023-10-17,7,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2023NE001661,C1ENCONDETM,CONCURSOS AS ESCOLAS MILITARES,0032,171502,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,BA,MG,167503,DECEX - GESTOR,C1EN,CON,DETM,10671.00,0.0,10671